# End to End Machine Learning - Medical Cost Predictions

Thank you for stopping by! This is my first project from scratch with no insight or input from course instructors so any feedback would be greatly appreciated to help me continue to expand my knowledge! I would also appreciate any insight on where everyone's go-to sources are to research algorithms and python code.

## Background

The United States spends significantly more money on healthcare than any other country around the world.  Spending is expected to continue to grow with The Centers for Medicare and Medicaid Services (CMS) projecting that by 2028, costs will climb to $6.2 trillion, or about $18,000 per person, and will represent about 20 percent of GDP. With an aging population and actual costs of services expected to increase as well it will be of great value for insurance companies, healthcare companies and patients to have an idea of where their future costs may be.  

## Machine Learning Process

In [ ]:
#! conda install pip
#! conda install ipykernel
#! pip install pandas-profiling

## Define Problem

We will be looking to answer the following question through prediction modeling: **What will a patient's future healthcare costs look like?**

## Raw Data Collection

The data we will be using for this project is an open-source data source from Kaggle regarding insurance costs.

To begin we will:

* Import our dataset
* Create our dataset into a dataframe

In [ ]:
#import necessary libraries for this work
import numpy as np #for dealing with arrays
import pandas as pd #for dealing with dataframes
import matplotlib.pyplot as plt #for visualization with matplotlib
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
%matplotlib inline
import seaborn as sns #for visualization with seaborn
from scipy import stats #for descriptive statistics such as pearson coef
import itertools

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier


import warnings
warnings.filterwarnings('ignore')


print('libraries imported')

In [ ]:
# Always good to set a seed for reproducibility
SEED = 7
np.random.seed(SEED)

In [ ]:
#import data set from csv and put into a dataframe
df = pd.read_csv('insurance.csv')
df.head()

## Data Pre-Processing 1 (AKA Data Wrangling)

In this step:

* Verify if any missing values
* Verify if any duplicates
* Verify format of the data and reformat if necessary
* Normalize the data if necessary
* Perform binning of the data if necessary

In [ ]:
import pandas_profiling
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="Basics of the Data")
profile.to_widgets()

In [ ]:
# variables rejected due to high correlation
rejected_features= list(profile.get_rejected_variables()) 
x_drop= df.drop(rejected_features,axis=1)
x_drop.shape

In [ ]:
#drop the duplicates
dup = df.drop_duplicates(keep="first", inplace = True)
#verify duplicates were dropped
duplicates2 = df[df.duplicated()]
verify_duplicates = len(duplicates2)
verify_duplicates

This confirms there are no missing values and we dropped the duplicate rows, now I will verify the types of data in the dataframe:

In [ ]:
#verify types
df.dtypes

In [ ]:
#make sure dataframe looks good
df.head()

The dataframe looks good - we are ready to start!

## Exploratory Data Analysis

Here I will look at:

* Implementing descriptive statistics
* Group data as necessary
* Eliminate outliers if necessary
* Look at data correlations

In [ ]:
df.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8); # ; avoid having the matplotlib verbose informations

In [ ]:
bins = np.linspace(df.charges.min(), df.charges.max(), 10)
g = sns.FacetGrid(df, col="sex", hue="region", palette="Set1", col_wrap=2)
g.map(plt.hist, 'charges', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

From this we can see that the Southeast and Northeast have higher costs in both males and females 

Now lets take a look at age

In [ ]:
a1 = df['age']  
a1.describe()

Now lets take a look at outliers in the numeric columns age, charges, children

In [ ]:
sns.boxplot(x=df["charges"])
plt.show()

In [ ]:
# The interquartile range is the difference between the 75th percentile and the 25th percentile.
q1 = df['charges'].quantile(0.25)
q3 = df['charges'].quantile(0.75)
iqr = q3 - q1
print('iqr value is: ',iqr)

In [ ]:
#find upper and lower limits
lower_limit = q1 - 1.5 * iqr
upper_limit = q3 + 1.5 * iqr

print('lower bounds: ' + str(lower_limit))
print('upper bounds: ' + str(upper_limit))

In [ ]:
#find amount of outliers in charges
cols = ['charges']
((df[cols] < (q1 - 1.5 * iqr)) |(df[cols] > (q3 + 1.5 * iqr))).sum()

In [ ]:
sns.boxplot(x=df["age"])
plt.show()

In [ ]:
# The interquartile range is the difference between the 75th percentile and the 25th percentile.
q1a = df['age'].quantile(0.25)
q3a = df['age'].quantile(0.75)
iqr = q3a - q1a

#find upper and lower limits
lower_limit = q1a - 1.5 * iqr
upper_limit = q3a + 1.5 * iqr

#find amount of outliers in charges
cols = ['age']
((df[cols] < (q1a - 1.5 * iqr)) |(df[cols] > (q3a + 1.5 * iqr))).sum()

In [ ]:
sns.boxplot(x=df["children"])
plt.show()

In [ ]:
#find iqr
q1b = df['children'].quantile(0.25)
q3b = df['children'].quantile(0.75)
iqr = q3b - q1b

#find upper and lower limits
lower_limit = q1b - 1.5 * iqr
upper_limit = q3b + 1.5 * iqr

#find amount of outliers in charges
cols = ['children']
((df[cols] < (q1b - 1.5 * iqr)) |(df[cols] > (q3b + 1.5 * iqr))).sum()

Therefore we only have outliers in the charges column - 139 of them, and we will drop those outliers

In [ ]:
cols = ['charges'] 


df_new = df[~((df[cols] < (q1 - 1.5 * iqr)) |(df[cols] > (q3 + 1.5 * iqr))).any(axis=1)]
df_new.head()

### Now lets work on finding correlations

In [ ]:
df_new.corr()

Slight correlations between age and charges and bmi and charges but none of the correlations give us anything really definitive, so lets take a look at Pearson Coefficients

p-value is < 0.001: we say there is strong evidence that the correlation is significant. \
p-value is < 0.05: there is moderate evidence that the correlation is significant. \
p-value is < 0.1: there is weak evidence that the correlation is significant. \
p-value is > 0.1: there is no evidence that the correlation is significant.

In [ ]:
pearson_coef, p_value = stats.pearsonr(df_new['age'], df_new['charges'])
print("The Pearson Correlation Coefficient is", pearson_coef, " with a P-value of P =", p_value)

In [ ]:
pearson_coef, p_value = stats.pearsonr(df_new['bmi'], df_new['charges'])
print("The Pearson Correlation Coefficient is", pearson_coef, " with a P-value of P =", p_value)

In [ ]:
pearson_coef, p_value = stats.pearsonr(df_new['children'], df_new['charges'])
print("The Pearson Correlation Coefficient is", pearson_coef, " with a P-value of P =", p_value)

In [ ]:
# Age as potential predictor variable of cost
sns.regplot(x="age", y="charges", data=df)
plt.ylim(0,)

In [ ]:
# Children as potential predictor variable of cost
sns.regplot(x="children", y="charges", data=df)
plt.ylim(0,)

In [ ]:
# BMI as potential predictor variable of cost
sns.regplot(x="bmi", y="charges", data=df)
plt.ylim(0,)

In [ ]:
df1=pd.DataFrame(df, columns=["bmi","charges"])
df1
sns.histplot(
    df1, x="bmi", y="charges",
    bins=30, discrete=(False), log_scale=(False),
    cbar=True, cbar_kws=dict(shrink=.75),
)

In [ ]:
sns.boxplot(x="children", y="charges", data=df)

In [ ]:
df.dtypes

In [ ]:
df.head()

## Split Data to Training and Testing Sets

### Handle categorical features

In [ ]:
categorical_list = []
numerical_list = []
for i in df.columns.tolist():
    if df[i].dtype=='object':
        categorical_list.append(i)
    else:
        numerical_list.append(i)
print('Number of categorical features:', str(len(categorical_list)))
print('Number of numerical features:', str(len(numerical_list)))

In [ ]:
for col in ['region', 'sex', 'smoker']:
    df[col] = df[col].astype('category')
    
df.dtypes

In [ ]:
df= pd.get_dummies(df)
df.head()

In [ ]:
y=df.charges
X=df.drop('charges',axis=1)
feature_name = X.columns.tolist()


x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=df[['sex_female','sex_male','age']])
x_test.shape

## Data Pre-processing 2 

Here I will perform:

* Feature Selection
* Feature Scaling
* Dimensional Reduction if necessary

### Lets pick some features

In [ ]:
def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-100:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

cor_support, cor_feature = cor_selector(X, y)
print(str(len(cor_feature)), 'selected features')

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["charges"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features

In [ ]:
print(df[["smoker_no","smoker_yes"]].corr())

In [ ]:
import statsmodels.api as sm

#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(y,X_1).fit()
model.pvalues

In [ ]:
#Backward Elimination
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
from sklearn.feature_selection import RFE

model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 7)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(x_train,y_train)
    X_test_rfe = rfe.transform(x_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
cols = list(X.columns)
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 10)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

## Select Machine Learning Algorithm

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge

from sklearn.pipeline import Pipeline
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('reduce_dim', PCA()),
        ('regressor', Ridge())
        ])

In [ ]:
pipe = pipe.fit(x_train, y_train)
print('Testing score: ', pipe.score(x_test, y_test))

In [ ]:
print(pipe.steps[1][1].explained_variance_)

In [ ]:
import numpy as np
n_features_to_test = np.arange(1, 11)

alpha_to_test = 2.0**np.arange(-6, +6)
params = {'reduce_dim__n_components': n_features_to_test,\
              'regressor__alpha': alpha_to_test}

In [ ]:
from sklearn.model_selection import GridSearchCV
gridsearch = GridSearchCV(pipe, params, verbose=1).fit(x_train, y_train)
print('Final score is: ', gridsearch.score(x_test, y_test))

Tuning the Pipeline

In [ ]:
scalers_to_test = [StandardScaler(), RobustScaler(), QuantileTransformer()]

In [ ]:
params = {'scaler': scalers_to_test,
        'reduce_dim__n_components': n_features_to_test,\
        'regressor__alpha': alpha_to_test}

In [ ]:
params = [
        {'scaler': scalers_to_test,
         'reduce_dim': [PCA()],
         'reduce_dim__n_components': n_features_to_test,\
         'regressor__alpha': alpha_to_test},

        {'scaler': scalers_to_test,
         'reduce_dim': [SelectKBest(f_regression)],
         'reduce_dim__k': n_features_to_test,\
         'regressor__alpha': alpha_to_test}
        ]

In [ ]:
gridsearch = GridSearchCV(pipe, params, verbose=1).fit(x_train, y_train)
print('Final score is: ', gridsearch.score(x_test, y_test))

In [ ]:
gridsearch.best_params_

We will take a look at the following models:

* Simple linear regression 
* Multiple linear regression 
* Polynomial regression
* Support Vector 
* Decision Tree
* Random Forest

### Simple Linear

In [ ]:
lr = LinearRegression().fit(x_train,y_train)
preds = lr.predict(x_test)


print('Score: ', lr.score(x_test, y_test))
print('MSE: ', mean_squared_error(y_test, preds)) 
print('R2:', r2_score(y_test, preds)) 


### Multiple Linear

In [ ]:
x = df.drop(['charges'], axis = 1)
y = df.charges

x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = 0)
lr = LinearRegression().fit(x_train,y_train)

y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)

print(lr.score(x_test,y_test))

Not bad but let's try something more complex

### Polynomial

In [ ]:
X = df.drop(['charges','sex_female','sex_male','children'], axis = 1)
Y = df.charges



quad = PolynomialFeatures (degree = 2)
x_quad = quad.fit_transform(X)

x_train,x_test,y_train,y_test = train_test_split(x_quad,Y, random_state = 1)

plr = LinearRegression().fit(x_train,y_train)

y_train_pred = plr.predict(x_train)
y_test_pred = plr.predict(x_test)

print(plr.score(x_test,y_test))
print('MSE train data: %.3f, MSE test data: %.3f' % (
mean_squared_error(y_train,y_train_pred),
mean_squared_error(y_test,y_test_pred)))
print('R2 train data: %.3f, R2 test data: %.3f' % (
r2_score(y_train,y_train_pred),
r2_score(y_test,y_test_pred)))

### Support Vector

In [ ]:
from sklearn.svm import SVR 

supportvectormodel = SVR(kernel='linear')
svm = supportvectormodel.fit(x_train,y_train)

y_pred_test = svm. predict (x_test)

print(supportvectormodel.score(x_test, y_test))
print('MSE:', (mean_squared_error(y_test, y_pred_test)))
print('R2:', (r2_score(y_test, y_pred_test)))

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor 

# create a regressor object
decT = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
dect_predict = decT.fit(x_train, y_train)

y_dec_test = dect_predict.predict(x_test)

print(dect_predict.score(x_test, y_test))
print('MSE:', (mean_squared_error(y_test, y_dec_test)))
print('R2:', (r2_score(y_test, y_dec_test)))

### Random Forest

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators = 100,
                              criterion = 'mse',
                              random_state = 1,
                              n_jobs = -1)
frst = forest.fit(x_train,y_train)
forest_train_pred = forest.predict(x_train)
forest_test_pred = forest.predict(x_test)


print(plr.score(x_test,y_test))
print('MSE train data: %.3f, MSE test data: %.3f' % (
mean_squared_error(y_train,forest_train_pred),
mean_squared_error(y_test,forest_test_pred)))
print('R2 train data: %.3f, R2 test data: %.3f' % (
r2_score(y_train,forest_train_pred),
r2_score(y_test,forest_test_pred)))

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(forest_train_pred,forest_train_pred - y_train,
          c = 'black', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
plt.scatter(forest_test_pred,forest_test_pred - y_test,
          c = 'b', marker = 'o', s = 35, alpha = 0.5,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend(loc = 'upper left')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'red')
plt.show()

## Conclusion

Result & Implication of the result

From all the models that we ran it was clear that the random forest was by far the most accurate with an R2 on the train data of 0.975 and an overall score of 0.833